In [395]:
from sklearn.datasets import load_boston
from sklearn.metrics import explained_variance_score
import numpy as np

x = load_boston().data
y = load_boston().target

In [396]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=29)

x_train.shape

(379, 13)

In [399]:
def polynom(m, deg):
    n_rows, n_cols = m.shape[0], m.shape[1]
    nm = np.zeros((m.shape[0],n_cols*deg))
    for d in range(deg):
        for c in range(n_cols):
            for r in range(n_rows):
                nm[r][n_cols*d+c] = m[r][c]**(d+1)
    return nm

In [520]:
def scaling(X):
    for c in range(X.shape[1]):
        X[:,c] = [(i - np.mean(X[:,c])) / (max(X[:,c]) - min(X[:,c])) for i in X[:,c]]
    return X

In [415]:
def predict(X, theta):
    return np.c_[np.ones(len(X)), scaling(X)] @ theta

In [522]:
def normalEq(X, y):
    X = np.c_[np.ones(len(X)), scaling(X)]
    return (np.linalg.inv(X.T @ X) @ X.T @ y).T

theta = normalEq(x_train, y_train)

print(theta)

preds = predict(x_test, theta)

print(f'\nNorm Eq EVS: {explained_variance_score(preds, y_test)}')

[ 22.61794195  -6.88081299   5.31800698   1.00151806   3.43338847
  -7.19704237  18.47655796  -0.2396351  -15.02450125   7.41163346
  -6.77932357  -9.07260822   4.38736543 -19.97598678]

Norm Eq EVS: 0.7416264097926808


$X^{'}X=\begin{bmatrix} 7 & 38.5\\  38.5& 218.75 \end{bmatrix}$

In [450]:
def BGD(X, y, n_iter, eta):
    X = np.c_[np.ones(len(X)), scaling(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    for i in range(n_iter):
        for c in range(X.shape[1]):
            nX = X[:,c]
            nT = theta[:,c]
            gradients = ((X @ theta.T) - y.T).T @ nX
            theta[:,c] = nT - eta * gradients
    return theta.T

theta = BGD(polynom(x_train, 2), y_train, 1000, 0.001)

print(theta)

preds = predict(polynom(x_test, 2), theta)

print(f'\nBGD EVS: {explained_variance_score(preds, y_test)}')

[[ 22.61794195]
 [ -5.75758067]
 [ -0.43118008]
 [ -3.39291134]
 [  1.81036381]
 [ -2.65118129]
 [  4.83968379]
 [ -5.05118642]
 [-11.66790218]
 [  5.56415289]
 [ -6.03086869]
 [ -5.95527633]
 [  5.60237287]
 [-27.13661868]
 [ -2.11845673]
 [  4.92986586]
 [  4.97484442]
 [  1.70369378]
 [ -4.59880081]
 [ 15.5191705 ]
 [  4.55472592]
 [ -0.40678727]
 [  1.56271903]
 [ -0.98845325]
 [ -2.80162931]
 [ -1.17034739]
 [ 10.84857532]]

BGD EVS: 0.7738285882557906


In [449]:
def learning_schedule(t):
    return 5 / (t + 50)

def SGD(X, y, n_epoch, eta):
    X = np.c_[np.ones(len(X)), scaling(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    m = X.shape[0]
    for epoch in range(1, n_epoch):
        for n in range(m):
            for c in range(X.shape[1]):
                r = np.random.randint(0, m-1)
                xi, yi = X[r:r+1], y.T[r:r+1]
                gradients = ((xi @ theta.T) - yi.T).T @ xi[:,c]
#                 eta = eta/epoch
                theta[:,c] = theta[:,c] - (gradients * eta)
    return theta.T

theta = SGD(polynom(x_train, 2), y_train, 50, 0.01)

print(theta)

preds = predict(polynom(x_test, 2), theta)

print(f'\nSGD EVS: {explained_variance_score(preds, y_test)}')

[[ 22.05000418]
 [ -4.0973114 ]
 [  0.07988003]
 [ -1.52733234]
 [  1.34766906]
 [ -4.52854114]
 [  7.1313255 ]
 [ -4.63958019]
 [ -8.10832683]
 [  1.52855544]
 [ -4.27495986]
 [ -6.51251444]
 [  2.77270949]
 [-19.65316074]
 [ -2.19747073]
 [  4.59488615]
 [  3.40810127]
 [  2.23792493]
 [ -2.23752565]
 [ 14.16033791]
 [  3.63684438]
 [ -3.95404031]
 [  3.50733473]
 [ -1.0890194 ]
 [ -2.56232618]
 [  1.07388276]
 [  2.45030504]]

SGD EVS: 0.7563749488752401


In [448]:
def MBGD(X, y, n_epoch, batch_size, eta):
    X = np.c_[np.ones(len(X)), scaling(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    m = X.shape[0]
    for epoch in range(n_epoch):
        shuffi = np.random.permutation(m)
        xs, ys = X[shuffi], y.T[shuffi]
        for i in range(0, m, batch_size):
            for c in range(X.shape[1]):
                r = np.random.randint(0, m-1)
                xi, yi = X[i:i+batch_size], y.T[i:i+batch_size]
                gradients = ((xi @ theta.T) - yi).T @ xi[:,c]
                theta[:,c] = theta[:,c] - (gradients * eta)
    return theta.T
        
theta = MBGD(polynom(x_train, 2), y_train, 100, 20, 0.01)

print(theta)

preds = predict(polynom(x_test, 2), theta)

print(f'\n MBGD EVS: {explained_variance_score(preds, y_test)}')

[[ 22.82408452]
 [ -6.21896215]
 [ -0.79063871]
 [ -3.14689727]
 [  1.58526413]
 [ -2.54600289]
 [  4.9726204 ]
 [ -4.6874097 ]
 [-11.82461518]
 [  6.03388815]
 [ -5.66038599]
 [ -5.98729169]
 [  5.67994645]
 [-26.88310055]
 [ -1.86282954]
 [  5.22344137]
 [  4.80385485]
 [  1.91052752]
 [ -4.95915987]
 [ 15.65314259]
 [  4.40842578]
 [ -0.53721881]
 [  1.44235291]
 [ -1.2888374 ]
 [ -2.77773447]
 [ -1.1614262 ]
 [ 10.57271582]]

 MBGD EVS: 0.7706085328989566


In [454]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

polybig_features = PolynomialFeatures(degree=2, include_bias=False)
std_scaler = StandardScaler()
lin_reg = LinearRegression()
polynomial_regression = Pipeline([
        ("poly_features", polybig_features),
        ("std_scaler", std_scaler),
        ("lin_reg", lin_reg),
    ])
polynomial_regression.fit(x_train, y_train)
preds = polynomial_regression.predict(x_test)

In [455]:
m = LinearRegression()

m.fit(polynom(x_train, 2), y_train)

preds = m.predict(polynom(x_test, 2))

print('weights: ')
print(m.coef_)
print('Intercept: ')
print(m.intercept_)

print(f'\n Scikit learn lin reg EVS: {explained_variance_score(y_test, preds)}')

weights: 
[ -4.82353170e-01  -6.11811058e-02  -1.03006200e-01   1.52751036e+00
  -2.66093614e+01  -1.72065010e+01  -5.67860772e-02  -2.20943742e+00
   5.80765699e-01  -2.01868784e-02  -6.10801742e+00   2.14283984e-02
  -1.54975811e+00   4.96884001e-03   8.07538384e-04   6.18487450e-03
   1.52751037e+00   3.81582232e+00   1.54888132e+00   5.40426439e-04
   1.03788287e-01  -4.23190591e-03   6.78223968e-06   1.49536968e-01
  -3.09227294e-05   2.80283423e-02]
Intercept: 
164.391664059
Scikit learn lin reg EVS: 0.8153831306538188
